# Overview

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import csv
import sys
import seaborn as sns
from scipy import stats

%matplotlib inline
import matplotlib.pyplot as plt

#setting pandas display options
pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', 199)  # or 199

BASE_DIR = "/home/jovyan/work/"

# Main datasets

## New York Times-provided per-county COVID case loads

The NYTCCL has a few points of missing data. 

### Reporting start time and frequency

1. Different counties started tracking their COVID caseloads at different dates, so they are missing data before they started measuring and reporting
2. Different counties report at different intervals. Whereas some counties reported daily counts, others reported only a few times a week
3. Some counties had irregular gaps up to a few days between measurements

#### Response

For the first case, we can make either disregard those dates, or assume that their case loads were zero. In our analyses, we will likely be interested only in the cases at later dates during the pandemic or in growth rates. It is unlikely that our choice to disregard these dates or mark them as zero will have a large impact on our analysis. 

For the second and third scenarios, we can impute values via either

- Linear interpolation, or
- Assuming no change from the previous measurement

Judging from the plots shown below, the plots appear rather smooth, so there isn't much to be gained by interpolating values. We will assume they have not changed from the previous measurement. We can perform our analysis with both the linearly interpolated and left-alone values to compare in the end. 

### California cases without county name and FIPS number

The second category of missing data here is that there are a handful (a dozen or so) cases that are not associated with any county in California. These dozen or so cases are a only a miniscule fraction of the nearly million cases in California, so we will be disregarding them and simply dropping those rows from the dataset.

In [ ]:
read_csv_parameters = {
    'parse_dates': ['date'],
    'dtype': {
        'county': 'string',
        'state': 'string',
        'fips': 'UInt64',
        'cases': 'UInt64',
        'deaths': 'UInt64'
    }
}

nytccl = pandas.read_csv(BASE_DIR + 'data-sets/raw-datasets/us-counties.csv', **read_csv_parameters)
nytccl = nytccl[ nytccl['state'] == 'California' ] 

nytccl.drop(nytccl[ nytccl['fips'].isnull() ].index, inplace=True)

nytccl = nytccl.reset_index() \
    .drop(columns=['index'])

lastDateInDataset = nytccl.tail(1)['date'].iloc[0]

print(nytccl[ nytccl['date'] == lastDateInDataset ].sum())

for title, group in nytccl.groupby(['county']):
    group.plot(x = 'date', y = 'cases', title = title)

In [ ]:
nytccl.tail(1)['date'].iloc[0]

## Treasury department PPP loans dataset

### Loans under $150,000

Edits by: Karina Lopez

Input fname: ca_PPP_150kunder.csv
<br>Output fname: ca_ppp_loans_under_150k.csv

<br>*The PPP loan dataset is divided into two separate datasets. This dataset only contains PPP loans up to 150K.*

**Datacleaning process:**
- Column names were changed based on the standards we set as a team
- Proportion of missing data was checked for each column. Columns with over 80% of data missing were identified and added to read me file
- Strings were fixed to remove unnecessary characters
- Data was converted to approriate data types



In [ ]:
os.chdir(BASE_DIR + 'data-sets/raw-datasets/120120 Paycheck Protection Program Data/')

# Load in your datasets
PPP_df = pd.read_csv('01 PPP sub 150k through 112420.csv')


In [ ]:
# filter to only include CA data

PPP_df = PPP_df[PPP_df['State'] == 'CA']

In [ ]:
# Replace unspecified data w/ NAs
PPP_df['RaceEthnicity'] = PPP_df['RaceEthnicity'].replace('Unanswered', np.NaN)
PPP_df['Gender'] = PPP_df['Gender'].replace('Unanswered', np.NaN)
PPP_df['Veteran'] = PPP_df['Veteran'].replace('Unanswered', np.NaN)

In [ ]:
print(PPP_df.isnull().sum() * 100 / len(PPP_df))

In [ ]:
print(PPP_df.isnull().sum())

In [ ]:
# since zipcode is really important for pairing to our county data, check if you can fill in the missing zipcode w/ city name
PPP_df_NA = PPP_df[PPP_df.Zip.isnull()]
display(PPP_df_NA)

# Since we do not have business names or any city names, we will keep the missing data in their own category

In [ ]:
# convert everything to lowercase strings
PPP_df = PPP_df.apply(lambda x: x.astype(str).str.lower())


In [ ]:
PPP_df.columns

In [ ]:
# rename columns so thet can be matched to other datasets

PPP_df.rename(columns = {'LoanAmount':'loan_amount', 'City':'city', 'Zip':'zipcode', 'NAICSCode':'NAICS_code', 'BusinessType':'business_type',
                         'JobsReported':'jobs_reported', 'DateApproved':'date_approved', 'Lender':'lender', 'State': 'state',
                         'RaceEthnicity':'race_ethnicity', 'Gender':'gender', 'Veteran':'veteran', 'NonProfit':'non_profit'}, inplace=True)



In [ ]:
PPP_df.head(n = 5)

In [ ]:
PPP_df.loan_amount = pd.to_numeric(PPP_df.loan_amount, errors = 'coerce')
PPP_df.zipcode = pd.to_numeric(PPP_df.zipcode, errors = 'coerce')
PPP_df.NAICS_code = pd.to_numeric(PPP_df.NAICS_code, errors = 'coerce')
PPP_df.jobs_reported = pd.to_numeric(PPP_df.jobs_reported, errors = 'coerce')


In [ ]:
# Long workaround to put in nan for missing data
PPP_df.loan_amount = PPP_df.loan_amount.fillna(-1)
PPP_df.loan_amount = PPP_df.loan_amount.astype(int)
PPP_df.loan_amount = PPP_df.loan_amount.astype(str)
PPP_df.loan_amount = PPP_df.loan_amount.replace('-1', np.nan)

PPP_df.zipcode = PPP_df.zipcode.fillna(-1)
PPP_df.zipcode = PPP_df.zipcode.astype(int)
PPP_df.zipcode = PPP_df.zipcode.astype(str)
PPP_df.zipcode = PPP_df.zipcode.replace('-1', np.nan)

PPP_df.NAICS_code = PPP_df.NAICS_code.fillna(-1)
PPP_df.NAICS_code = PPP_df.NAICS_code.astype(int)
PPP_df.NAICS_code = PPP_df.NAICS_code.astype(str)
PPP_df.NAICS_code = PPP_df.NAICS_code.replace('-1', np.nan)

PPP_df.jobs_reported = PPP_df.jobs_reported.fillna(-1)
PPP_df.jobs_reported = PPP_df.jobs_reported.astype(int)
PPP_df.jobs_reported = PPP_df.jobs_reported.astype(str)
PPP_df.jobs_reported = PPP_df.jobs_reported.replace('-1', np.nan)


In [ ]:
print(PPP_df.dtypes)

In [ ]:
print(PPP_df.shape)
PPP_df.head(n = 5)


In [ ]:
PPP_df.business_type.value_counts()

In [ ]:
PPP_df.columns = PPP_df.columns.str.lower()

In [ ]:
# save as a new "cleaned up" csv
os.chdir(BASE_DIR + 'data-sets/clean-datasets/')
PPP_df.to_csv('ca_ppp_loans_under_150k.csv', index = False)

### Loans over $150,000

This dataset contains PPP loans over 150k.

Summary of Data-cleaning process:

- Change column names according to naming convention established by team (lowercase, all spaces underscored)
- Find rows where State column is null, then check CD column for any rows that are CA businesses
- Filter rows where State is CA, add the rows from the previous step, then use this new dataframe since we are only looking at CA businesses
- Change all strings in dataset to be lowercase
- Change column values to appropriate datatypes
- Check percent of missing values for all columns

In [ ]:
new_ppp = pd.read_csv(BASE_DIR + 'data-sets/raw-datasets/120120 Paycheck Protection Program Data/150k plus PPP through 112420.csv')
new_ppp.head()

In [ ]:
#change the column names to lowercase and add underscores
new_ppp.columns = new_ppp.columns.str.lower()
new_ppp = new_ppp.rename(columns={"loanamount": "loan_amount", "businessname": "business_name", 
                         "naicscode":"naics_code", "businesstype":"business_type",
                        "raceethnicity":"race_ethnicity", "nonprofit":"non_profit",
                         "jobsreported":"jobs_reported", "dateapproved":"date_approved", "zip":"zipcode"})
new_ppp.head()

In [ ]:
#find all rows where state is null
new_ppp[new_ppp['state'].isnull()]

In [ ]:
#looking at the rows with no given state, in the cd column we see that one row definitely belongs to CA
#save this row to add later 
new_ca_row = new_ppp[new_ppp['business_name']=='KIRTLEY CONSTRUCTION INC']

#filter dataset to to only include CA businesses
#then append the row above to dataset
new_ppp = new_ppp[new_ppp['state'] == 'CA']
new_ppp_150_plus = pd.concat([new_ppp, new_ca_row]).reset_index(drop=True)
new_ppp_150_plus

In [ ]:
#since the appended row still has NaN as its state value, fill it in with CA
new_ppp_150_plus['state'] = new_ppp_150_plus['state'].fillna('CA')

In [ ]:
#change all strings in dataset to be lowercase
new_ppp_150_plus = new_ppp_150_plus.apply(lambda x: x.astype(str).str.lower())

In [ ]:
#change date_approved column to be in datetime format
new_ppp_150_plus['date_approved'] = pd.to_datetime(new_ppp_150_plus['date_approved'])

In [ ]:
#change loan amount type to int
new_ppp_150_plus['loan_amount'] = new_ppp_150_plus['loan_amount'].astype(float).astype(int)

In [ ]:
#replace all 'nan' values with NaN so it is a python-recognized missing value
new_ppp_150_plus = new_ppp_150_plus.replace('nan', np.NaN)

In [ ]:
#change zipcode, naics_code and jobs_reported columns types to integer without removing NA values
new_ppp_150_plus['zipcode'] = pd.to_numeric(new_ppp_150_plus['zipcode'], errors='coerce').astype('Int64')
new_ppp_150_plus['naics_code'] = pd.to_numeric(new_ppp_150_plus['naics_code'], errors='coerce').astype('Int64')
new_ppp_150_plus['jobs_reported'] = pd.to_numeric(new_ppp_150_plus['jobs_reported'], errors='coerce').astype('Int64')

In [ ]:
#print % of null values for each column at this point
print(new_ppp_150_plus.isnull().sum() * 100 / len(new_ppp_150_plus))

In [ ]:
#for columns that have 'unanswered' as a column value, replace with NaN to see how this changes null %
new_ppp_150_plus['race_ethnicity'] = new_ppp_150_plus['race_ethnicity'].replace('unanswered', np.NaN)
new_ppp_150_plus['gender'] = new_ppp_150_plus['gender'].replace('unanswered', np.NaN)
new_ppp_150_plus['veteran'] = new_ppp_150_plus['veteran'].replace('unanswered', np.NaN)
print(new_ppp_150_plus.isnull().sum() * 100 / len(new_ppp_150_plus))

In [ ]:
new_ppp_150_plus.head()

In [ ]:
new_ppp_150_plus.to_csv(BASE_DIR + 'data-sets/clean-datasets/ca_new_ppp_loans_150_plus_clean.csv')

# Supplementary datasets

## California zipcodes and counties

Edits by: Karina Lopez

Input fname: ca_census_county_zipcodes.csv
<br>Output fname: ca_zipcodes.csv

<br>*The zipcode dataset contains all zipcodes and cities in California. This data set can be used as a merging key*

**Datacleaning process:**
- Completely empty rows (no data in any columns) were removed from the dataset
- County names that were missing were filled in by searching for the county the city entry belomged to in Google
- Data types were converted to the appropriate typing (e.g., zipcodes are integers)
- Strings were all converted to lowercase
- Column names were all converted to lowercase to abide by dataset standards set by the team (eases merging process)


In [ ]:
os.chdir(BASE_DIR + 'data-sets/raw-datasets/')

# Load in your datasets
zipcodes_df = pd.read_csv('ca_census_county_zipcodes.csv')

In [ ]:
# check for missing values
print(zipcodes_df.isnull().sum())

# Show rows w/ missing values
zipcodes_df_NA = zipcodes_df[zipcodes_df.isnull().any(axis=1)]
display(zipcodes_df_NA )


In [ ]:
# Manually looked up counties () and updated their values
# La Quinta: https://en.wikipedia.org/wiki/La_Quinta,_California (riverside)
# Oceano: https://en.wikipedia.org/wiki/Oceano,_California (san luis obispo)

zipcodes_df.at[746, 'county'] = 'Riverside'
zipcodes_df.at[747, 'county'] = 'Riverside'
zipcodes_df.at[1278, 'county'] = 'San Luis Obispo'

In [ ]:
# remove all NA values remaining
zipcodes_df = zipcodes_df.dropna() 
print(zipcodes_df.isnull().sum())

In [ ]:
# Check for duplicates
zipcodes_df.duplicated().sum()

In [ ]:
# convert everything to lowercase
zipcodes_df = zipcodes_df.apply(lambda x: x.astype(str).str.lower())

In [ ]:
zipcodes_df.zipcode = pd.to_numeric(zipcodes_df.zipcode)

In [ ]:
# convert zipcodes to integers
zipcodes_df.zipcode = zipcodes_df.zipcode.astype(int)

In [ ]:
# change column names for city in dataset
zipcodes_df.rename(columns = {'zipcode_name':'city'}, inplace = True)


In [ ]:
print(zipcodes_df.dtypes)
print(zipcodes_df.shape)
zipcodes_df.head(n = 5)

In [ ]:
zipcodes_df.zipcode_type.value_counts()

In [ ]:
# save as a new "cleaned up" csv
os.chdir(BASE_DIR + 'data-sets/clean-datasets/')
zipcodes_df.to_csv('ca_zipcodes.csv', index = False)

## Census urban and rural populations by county

Edits by: Karina Lopez

Input fname: ca_census_rural.csv
<br>Output fname: ca_county_population.csv

<br>*The census population dataset contains population counts for each county. It idenitifies population counts and percentages for both rural and urban areas in each county.*

**Datacleaning process:**
- Column names were changed based on the standards we set as a team
- Strings were fixed to remove unnecessary characters
- Data was converted to approriate data types


In [ ]:
os.chdir(BASE_DIR + 'data-sets/raw-datasets/')

# Load in your datasets
census_df = pd.read_csv('ca_census_rural.csv')


In [ ]:
# check for missing values
print(census_df.isnull().sum())

In [ ]:
# change column names in dataset
census_df.rename(columns = {'2015 GEOID': 'GEOID', 'State': 'state', '2010 Census Urban Population':'urban_population', '2015 Geography Name':'county', '2010 Census Total Population':'total_population',
                             '2010 Census Rural Population':'rural_population', '2010 Census Percent Rural':'rural_percent'}, inplace=True)



In [ ]:
# remove strings from each county column
census_df['county'] = census_df['county'].str.rstrip(', California')
census_df['county'] = census_df['county'].str.rstrip('County')

In [ ]:
# convert everything to lowercase
census_df = census_df.apply(lambda x: x.astype(str).str.lower())

In [ ]:
# remove whitespace
census_df['county'] = census_df['county'].apply(lambda x:x.strip())


In [ ]:
print(census_df.dtypes)

# some numerical columns are strings (object). We need to remove commas to convert them to number columns
census_df.replace(',','', regex = True, inplace = True)


In [ ]:
# convert number data to integers and floats
census_df.total_population = pd.to_numeric(census_df.total_population)
census_df.urban_population = pd.to_numeric(census_df.urban_population)
census_df.rural_population = pd.to_numeric(census_df.rural_population)
census_df.rural_percent = pd.to_numeric(census_df.rural_percent)
census_df.GEOID = pd.to_numeric(census_df.GEOID)


In [ ]:
print(census_df.dtypes)

In [ ]:
print(census_df.shape)
census_df.head(n = 5)

In [ ]:
census_df.columns = census_df.columns.str.lower()

In [ ]:
# save as a new "cleaned up" csv
os.chdir(BASE_DIR + 'data-sets/clean-datasets/')
census_df.to_csv('ca_county_population.csv', index = False)


## Industry classification codes

This dataset contains a set of codes and titles used to uniquely identify each industry in the United States.

Summary of Data-cleaning process:

- Drop column with all missing values
- Remove row with all NaN values
- Change column names according to naming convention established by team (lowercase, all spaces underscored)
- Lowercase title column
- Change NAICS code column type to int

In [ ]:
#import industry codes csv 
industry_codes = pd.read_csv(BASE_DIR + 'data-sets/raw-datasets/industry_codes_raw.csv')
industry_codes.head()

In [ ]:
#see if there are any values that aren't missing in the 'Unnamed: 2' column
industry_codes[industry_codes['Unnamed: 2'].notna()]

In [ ]:
#number of missing values in 'Unnamed: 2' column out of 1058 total rows
industry_codes['Unnamed: 2'].isna().sum()

In [ ]:
#skip the first row since it's all NaN values
#drop the 'Unnamed: 2' since it has all NaN values
industry_codes = industry_codes[1:].reset_index(drop=True)
industry_codes = industry_codes.drop('Unnamed: 2', axis = 1) 

industry_codes.head()

In [ ]:
#change column names to lowercase and add underscores
#change'2017_naics_title' to be all lowercase
#change'2017_naics_code' type to integer
industry_codes.columns = ['2017_naics_code', '2017_naics_title']
industry_codes['2017_naics_title'] = industry_codes['2017_naics_title'].str.lower()
industry_codes['2017_naics_code'] = industry_codes['2017_naics_code'].astype(int)


In [ ]:
industry_codes.head()

In [ ]:
industry_codes.isna().sum()

In [ ]:
industry_codes.to_csv(BASE_DIR + 'data-sets/clean-datasets/industry_codes_clean.csv')

## Employment by major industry sector

This dataset shows the number of jobs per major industry in 2009, 2019, and an estimate for 2029, in thousands.

Summary of Data-cleaning process:

- Drop rows and columns that contained all NaN values
- Convert first two rows of dataset into column names (were misplaced due to how they were imported)
- Change column names according to naming convention established by team (lowercase, all spaces underscored)
- Remove commas in numbers, change number types to float since they include important decimals
- Remove unnecessary parenthesis and numbers in the industry title column, make column lowercase

In [ ]:
#import industry csv
industry = pd.read_csv(BASE_DIR + 'data-sets/raw-datasets/industry_job_counts_raw.csv')

industry.head()

In [ ]:
#keep rows that aren't all NaN
industry = industry.copy()[:31]
industry = industry.drop([3, 5, 10, 25, 29]).reset_index(drop=True)

In [ ]:
#drop last four columns that contain all NaN values
industry = industry.drop(industry.iloc[:, 11:15], axis = 1) 

In [ ]:
#make first row the column names for more clarity, then delete that row since it has no information
industry.columns = industry.iloc[0]
industry = industry[1:].reset_index(drop=True)

industry.head()

In [ ]:
#change column names using an understandable naming convention
industry.columns = ['industry_title','thousands_of_jobs_2009', 'thousands_of_jobs_2019', 'thousands_of_jobs_2029', 'change_2009_2019', 'change_2019_2029', 'pct_distribution_2009','pct_distribution_2019', 'pct_distribution_2029', 'cmpd_annual_rate_of_change_2009_2019', 'cmpd_annual_rate_of_change_2009_2019']
industry.head()

In [ ]:
#drop the first row since it is now incorporated in the column names
industry = industry.copy()[1:]

industry.head()

In [ ]:
industry.columns[:5]

In [ ]:
#remove the commas for all column values that contain them
#keep as float since we are looking at thousands of jobs, so decimals might be important
for col in industry.columns[1:6]:
    industry[col]=industry[col].str.replace(',', '').astype(float)
industry.head()

In [ ]:
industry['industry_title'] = industry['industry_title'].str.replace('[(0-9)]', '', regex=True).str.lower()
industry.head()

In [ ]:
industry.isna().sum()

In [ ]:
industry.to_csv(BASE_DIR + 'data-sets/clean-datasets/industry_job_counts_clean.csv')

## Census population counts

## Labor Force and Unemployment Rate for California Counties

The Labor Force & Unemployment Rate for California Counties dataset provides totals for labor force, employed, and unemployed workers in California counties month to month, calculating an unemployment rate based on those numbers. 

The datacleaning process for this dataset wasn't too difficult. The dataset did not have any null values, only providing information on areas and years in which it was available. Once the dataset was formatted based on team parameters/standards, it was then adjusted in the following ways:

- The original dataset provided numbers not just on counties, but on other types of areas as well (large cities, municipal areas, the state as a whole). We isolated the data to only county-specific numbers.
- We dropped the 'status' column, which confirms whether the data is final or preliminary. Preliminary data only is relevant for the most recent month entry, which we believe will not have an effect on our analysis of the data.
- All counties, with the exception of Los Angeles County, had data that was not seasonally adjusted. Los Angeles provided both seasonally and non-seasonally adjusted data. To stay consistent, we removed the seasonally adjusted Los Angeles County entry, and subsequently removed the Seasonally Adjusted column. 
- Given that we are interested in the impact of the PPP Loans, which occurred over this past year, we limited the size of the dataset to entries from 2020 only.

In [ ]:
df = pd.read_csv(BASE_DIR + 'data-sets/raw-datasets/ca_employment_raw_sept2020.csv', sep=',')
counties_df = df.copy()
counties_df.head()

In [ ]:
#This code is to format the df to team-specified standards
counties_df.columns = counties_df.columns.str.lower()
counties_df.rename(columns = lambda x: x.strip(), inplace=True)
counties_df.columns = counties_df.columns.str.replace(' ', '_')
counties_df.rename(columns={'seasonally_adjusted_(y/n)':'seasonally_adjusted', 'status_(preliminary_/_final)':'status'}, inplace=True)
counties_df['date'] = pd.to_datetime(counties_df['date'])
cols_to_change = ['area_type', 'month', 'seasonally_adjusted', 'status']

for col in cols_to_change:
    counties_df[col] = counties_df[col].str.lower().str.replace(' ',  '_')
    
counties_df['area_name'] = counties_df['area_name'].str.lower()

In [ ]:
#isolating county data only
counties_df = counties_df[counties_df['area_type'] == 'county']

In [ ]:
#dropping unnecessary columns
counties_df.drop(columns=['area_type', 'status'], inplace=True)

In [ ]:
#Isolating 2020 data only
counties_2020_df = counties_df[counties_df['year'] == 2020]
counties_2020_df.reset_index(drop=True, inplace=True)

In [ ]:
#Removing LA County Seasonally adjusted data, seasonally adjusted column
counties_2020_df = counties_2020_df[counties_2020_df.seasonally_adjusted != 'y']
counties_2020_df.drop(columns=['seasonally_adjusted'], inplace=True)

In [ ]:
counties_2020_df.head()

## United States Temporary Business Closures

The data in this set are entirely complete.

In [ ]:
api = pd.read_csv('/home/jovyan/work/team-work/data-sets/raw-datasets/US_temporary_closures_by_State.csv')

<p>Let's go through the columns present in the dataframe:</p>

In [ ]:
api.columns

<p>We have the following columns</p>
<ol>
<li><strong>date</strong>: The date in which the information was retreived</li>
<li><strong>index</strong>: The degree of how closed the businesses were on that date. Permanently closed > 100</li>
<li><strong>close_reason</strong>: Closed or temporarily</li>
<li><strong>region</strong>: State</li>

Seeing that `date` is a date, and that `close_reason` and `region` are strings, I will make sure to properly read these with `read_csv`.

In [ ]:
api = pd.read_csv('/home/jovyan/work/team-work/data-sets/raw-datasets/US_temporary_closures_by_State.csv', 
	parse_dates=['date'], 
	dtype={
		'close_reason': 'string',
		'region': 'string'
	}
)
api.info()


The next thing I need to do is extract just California information.

In [ ]:
api_ca = api.loc[api.region == 'California']
api_ca

In [ ]:
api_ca.groupby(api_ca.date.dt.month).index.mean()

## California Taxable Sales by County

We are mainly using this dataset in order to draw California's counties. For some reason, however, the dataset repeats the shapes of the counties over and over again for every single year. To cut down on the size of this dataset, we have extracted only the 2019 data and saved the shapes from there. This reduced the size of the geoJSON file from ~307MiB to only ~16MiB.

In [ ]:
import inspect

from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import TerminalFormatter

def show_function(function):

	print(highlight(inspect.getsource(function), lexer, formatter))

def shoddy_import(path):

    _temp = __import__('.'.join(path))

    for submodule in path[1:]:

        _temp = getattr(_temp, submodule)

    return _temp

os.chdir(BASE_DIR)

explore = shoddy_import([ "scripts", "exploratory-analyses", "dacoda-project-scope", "explore" ])

lexer = PythonLexer()
formatter = TerminalFormatter()

show_function(explore.trim)
show_function(explore.dataFromScratch)
show_function(explore.grabData)

data = explore.grabData()